In [ ]:
import pandas as pd
#Extracted the geo-coordinates of imp POI's and ingesting them into a DataFrame

poi_list=[
    {"name": "Manyata Tech Park", "type": "Tech Park", "lat": 13.0416, "lon": 77.6139},
    {"name": "RMZ Ecospace", "type": "Tech Park", "lat": 12.9288, "lon": 77.6713},
    {"name": "Prestige Tech Park", "type": "Tech Park", "lat": 12.9469, "lon": 77.6974},
    {"name": "Bagmane World Technology Center", "type": "Tech Park", "lat": 12.9818, "lon": 77.6931},
    {"name": "Embassy TechVillage", "type": "Tech Park", "lat": 12.9239, "lon": 77.6835},
    {"name": "ITPB Whitefield", "type": "Tech Park", "lat": 12.9866, "lon": 77.7265},
    {"name": "Nexus Mall Koramangala", "type": "Mall", "lat": 12.9344, "lon": 77.6111},
    {"name": "Phoenix Marketcity", "type": "Mall", "lat": 12.9960, "lon": 77.6960},
    {"name": "VR Bengaluru", "type": "Mall", "lat": 12.9966, "lon": 77.6967},
    {"name": "UB City", "type": "Commercial", "lat": 12.9716, "lon": 77.5946},
    {"name": "Orion Mall", "type": "Mall", "lat": 13.0112, "lon": 77.5555},
    {"name": "Mantri Square Mall", "type": "Mall", "lat": 12.9918, "lon": 77.5708},
    {"name": "KSR Bengaluru Railway Station", "type": "Transit", "lat": 12.9781, "lon": 77.5697},
    {"name": "Kempegowda Bus Station", "type": "Transit", "lat": 12.9774, "lon": 77.5729},
    {"name": "Silk Board Junction", "type": "Transit/Hub", "lat": 12.9176, "lon": 77.6233},
    {"name": "St. John's Medical College Hospital", "type": "Hospital", "lat": 12.9340, "lon": 77.6190},
    {"name": "Manipal Hospital Old Airport Road", "type": "Hospital", "lat": 12.9592, "lon": 77.6475},
    {"name": "Vittal Mallya Road (Commercial)", "type": "Commercial", "lat": 12.9696, "lon": 77.5956}
]
df_poi=pd.DataFrame(poi_list)
df_poi.to_csv('geo_coordinates.csv', index=False)

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
encoded_password = "Bsa1986%40%21"
DB_HOST = "127.0.0.1:3306"
DB_NAME = "bengaluru_traffic" 
engine = create_engine(f"mysql+pymysql://root:{encoded_password}@{DB_HOST}/{DB_NAME}")

query="""
SELECT distinct area,road,latitude,longitude
FROM traffic_cleaned_geo
"""

corridor_df=pd.read_sql_query(query,engine)
poi_df=pd.read_csv('geo_coordinates.csv')

poi_df=poi_df.rename(columns={'lat':'latitude','lon':'longitude','name':'poi_name'})

corridor_df['corridor_key'] = corridor_df['area'] + ' - ' + corridor_df['road']

set(corridor_df['corridor_key']).intersection(set(poi_df['corridor']))

In [ ]:
#Distance Calculation using geodesic distance(km):
from geopy.distance import geodesic

rows = []

for _, c in corridor_df.iterrows():
    corridor_key = c['corridor_key']
    corridor_coord = (c['latitude'], c['longitude'])

    corridor_pois = poi_df[poi_df['corridor'] == corridor_key]
    
    if corridor_pois.empty:
        continue #skip this iteration

    for _, p in corridor_pois.iterrows():
        poi_coord = (p['latitude'], p['longitude'])

        distance_km = geodesic(corridor_coord, poi_coord).km

        rows.append({
            'area': c['area'],
            'road': c['road'],
            'corridor': corridor_key,
            'poi_name': p['poi_name'],
            'poi_type': p['poi_type'],
            'distance_km': round(distance_km, 2)
        })
dist_df = pd.DataFrame(rows)
dist_df

In [5]:
#POI's Bucketing:
def proximity_bucket(d):
    if d <= 1:
        return '≤ 1 km'
    elif d <= 2.5:
        return '≤ 2.5 km'
    elif d <= 5:
        return '≤ 5 km'
    else:
        return '> 5 km'

dist_df['proximity_bucket'] = dist_df['distance_km'].apply(proximity_bucket)

#Final Spatial Analysis:
df_spatial=dist_df[['corridor','poi_name','poi_type','proximity_bucket']]

In [ ]:
#Visualization of poi's for every area and their proximity scale:
area_summary = (
    dist_df
    .groupby(['area', 'proximity_bucket'])
    .size()
    .reset_index(name='poi_count')
)
area_pivot = area_summary.pivot(
    index='area',
    columns='proximity_bucket',
    values='poi_count'
).fillna(0)

import matplotlib.pyplot as plt

# Ensure consistent bucket order
bucket_order = ['≤ 1 km', '≤ 2.5 km', '≤ 5 km', '> 5 km']
area_pivot = area_pivot.reindex(columns=bucket_order, fill_value=0)

# Plot
ax = area_pivot.plot(
    kind='bar',
    stacked=True,
    figsize=(12,6)
)

plt.title(
    "Spatial Exposure of Areas to Major POIs\n(Proximity-Based Distribution)",
    fontsize=14,
    weight='bold'
)
plt.xlabel("Area")
plt.ylabel("Number of POIs")

plt.legend(
    title="POI Proximity Bucket",
    bbox_to_anchor=(1.02, 1),
    loc='upper left'
)

plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.4)

plt.tight_layout()
plt.show()